In [1]:
from model.ctabgan import CTABGAN
from model.eval.evaluation import get_utility_metrics,stat_sim,privacy_metrics
import numpy as np
import pandas as pd
import glob

In [2]:
!pip install --upgrade jupyter ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install --upgrade setuptools wheel

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install setuptools_rust

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [6]:
!pip uninstall numpy -y

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5


In [7]:
!pip install numpy==1.23.5


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.23.5-cp39-cp39-macosx_10_9_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp39-cp39-macosx_10_9_x86_64.whl (18.1 MB)


In [8]:
!pip uninstall scipy scikit-learn -y

Found existing installation: scipy 1.5.4
Uninstalling scipy-1.5.4:
  Successfully uninstalled scipy-1.5.4
Found existing installation: scikit-learn 0.24.1
Uninstalling scikit-learn-0.24.1:
  Successfully uninstalled scikit-learn-0.24.1


In [9]:
!pip install scipy==1.5.4 scikit-learn==0.24.1

Defaulting to user installation because normal site-packages is not writeable
  Using cached scipy-1.5.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (2.0 kB)
  Using cached scikit_learn-0.24.1-cp39-cp39-macosx_10_13_x86_64.whl.metadata (9.7 kB)
Using cached scipy-1.5.4-cp39-cp39-macosx_10_9_x86_64.whl (29.1 MB)
Using cached scikit_learn-0.24.1-cp39-cp39-macosx_10_13_x86_64.whl (7.3 MB)


In [10]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [11]:
num_exp = 5
dataset = "Adult"
real_path = "Real_Datasets/Adult.csv"
fake_file_root = "Fake_Datasets"

In [ ]:
synthesizer =  CTABGAN(raw_csv_path = real_path,
                 test_ratio = 0.20,
                 categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income'], 
                 log_columns = [],
                 mixed_columns= {'capital-loss':[0.0],'capital-gain':[0.0]},
                 general_columns = ["age"],
                 non_categorical_columns = [],
                 integer_columns = ['age', 'fnlwgt','capital-gain', 'capital-loss','hours-per-week'],
                 problem_type= {"Classification": 'income'}) 

for i in range(num_exp):
    synthesizer.fit()
    syn = synthesizer.generate_samples()
    syn.to_csv(fake_file_root+"/"+dataset+"/"+ dataset+"_fake_{exp}.csv".format(exp=i), index= False)

  0%|                                                   | 0/150 [00:00<?, ?it/s]

In [ ]:
fake_paths = glob.glob(fake_file_root+"/"+dataset+"/"+"*")

In [ ]:
classifiers_list = ["lr","dt","rf","mlp"]
result_mat = get_utility_metrics(real_path,fake_paths,"MinMax",classifiers_list, test_ratio = 0.20)

result_df  = pd.DataFrame(result_mat,columns=["Acc","AUC","F1_Score"])
result_df.index = classifiers_list
result_df

In [ ]:
adult_categorical = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']
stat_res_avg = []
for fake_path in fake_paths:
    stat_res = stat_sim(real_path,fake_path,adult_categorical)
    stat_res_avg.append(stat_res)

stat_columns = ["Average WD (Continuous Columns","Average JSD (Categorical Columns)","Correlation Distance"]
stat_results = pd.DataFrame(np.array(stat_res_avg).mean(axis=0).reshape(1,3),columns=stat_columns)
stat_results

In [ ]:
priv_res_avg = []
for fake_path in fake_paths:
    priv_res = privacy_metrics(real_path,fake_path)
    priv_res_avg.append(priv_res)
    
privacy_columns = ["DCR between Real and Fake (5th perc)","DCR within Real(5th perc)","DCR within Fake (5th perc)","NNDR between Real and Fake (5th perc)","NNDR within Real (5th perc)","NNDR within Fake (5th perc)"]
privacy_results = pd.DataFrame(np.array(priv_res_avg).mean(axis=0).reshape(1,6),columns=privacy_columns)
privacy_results